## Import libraries

In [1]:
# import libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Load Data

In [3]:
tmdb_box = pd.read_csv('tmdb_box_merge.csv', sep = '\t')

## Data Processing

In [4]:
tmdb_box.head()

,Unnamed: 0,production_budget,domestic_gross_x,worldwide_gross,month,year,studio,genre_ids,original_language,popularity,vote_average,vote_count,date,month.1,year.1
0,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,5.0,2011.0,BV,"[12, 28, 14]",en,30.579,6.4,8571.0,2011-05-20,5.0,2011.0
1,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,6.0,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,5.0,2015.0,BV,"[28, 12, 878]",en,44.383,7.3,13457.0,2015-05-01,5.0,2015.0
3,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,12.0,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Star Wars Ep. VII: The Force Awakens,306000000.0,936662225.0,2.053311e+09,12.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Parsing Genre